# Lab 1: Quaternion interpolation

Define the quaternion interpolation function based on the Taylor method (pr, qr)=qpinter(P1, P2, lambda) that calculates the intermediate quaternion between $q_1$ (initial) and $q_2$ (final). 

The value $\lambda$ must satisfy $0 \le \lambda \le 1$, so that

- (p1, q1) = qpinter(P1, P2, 0) and 
- (p2, q2) = qpinter(P1, P2, 1)

## Relevant equations

Time law from initial time of the trajectory $t_i$ and final time of the trajectory $t_f$

$$\lambda(t) = \frac {t - t_i} {t_f - t_i} \quad \lambda \in [0, 1] \tag 1$$

Linerly interpolated transition:

$$p(t) = p_0 + \lambda(t) (p_1 - p_0) \tag 2$$

Quaternion as a rotation angle $\theta$ over an axis $n$:

$$Rot(n, \theta) = [w + \vec v] = [\cos \frac \theta 2 + \sin \frac \theta 2 \vec n ] \quad  n \in \mathbb{R} \quad v, n \in \mathbb{R}^3  \tag {3.1}$$

$$q = [w, \vec v] \quad \theta = \arccos(2) \quad \vec n = \frac {\vec v} {\sin \frac \theta 2} \tag {3.2}$$

3D rotation to go from $q_A$ to $q_B$:

$$q_A \cdot q_C = q_B \implies q_C = q_A^{-1} \cdot q_B = [w_c, \vec v_C] \tag 4$$

Interpolation of rotation:

$$\theta_{\lambda} (t) = \lambda(t) \theta \tag 5$$

Interpolation of the rotation quaternion:

$$q_{rot}(\lambda) = [w_{rot} (\lambda), \vec v_{rot} (\lambda)] \quad 
w_{rot} (\lambda) = \cos \frac {\theta_{\lambda}(\lambda)} {2} \quad \vec v_{rot}(\lambda) = \vec n \sin \frac {\theta_{\lambda}(\lambda)} {2} \tag 6$$

Interpolation quaternion $q_{\lambda}(\lambda)$:

$$q_{\lambda}(\lambda) = q_A \cdot q_{rot}(\lambda) \implies q_{\lambda} (0) = q_A \quad q_{\lambda} (1) = q_B \tag 7$$

Using Gemini we find a Python class to implement quaternions:

In [3]:
import numpy as np

class Quaternion:
    """
    A class representing a quaternion.

    Quaternions are often represented as q = a + bi + cj + dk, where a, b, c, d are real numbers,
    and i, j, k are the quaternion units satisfying:
    i² = j² = k² = -1
    ij = k, ji = -k
    jk = i, kj = -i
    ki = j, ik = -j
    """
    def __init__(self, w, x, y, z):
        """
        Initializes a quaternion.

        Args:
            w (float): The scalar part.
            x (float): The i component.
            y (float): The j component.
            z (float): The k component.
        """
        self.w = float(w)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __repr__(self):
        """
        String representation of the quaternion.
        """
        return f"Quaternion({self.w}, {self.x}, {self.y}, {self.z})"

    def __str__(self):
        """
        Human-readable string representation of the quaternion.
        """
        parts = []
        if self.w != 0:
            parts.append(f"{self.w}")
        if self.x != 0:
            parts.append(f"{self.x}i" if self.x == 1 else f"{self.x}i")
        if self.y != 0:
            parts.append(f"{self.y}j" if self.y == 1 else f"{self.y}j")
        if self.z != 0:
            parts.append(f"{self.z}k" if self.z == 1 else f"{self.z}k")

        if not parts:
            return "0"
        return " + ".join(parts).replace("+ -", "- ")

    def __add__(self, other):
        """
        Addition of two quaternions.
        """
        if isinstance(other, Quaternion):
            return Quaternion(self.w + other.w, self.x + other.x, self.y + other.y, self.z + other.z)
        else:
            raise TypeError("Can only add another Quaternion object.")

    def __sub__(self, other):
        """
        Subtraction of two quaternions.
        """
        if isinstance(other, Quaternion):
            return Quaternion(self.w - other.w, self.x - other.x, self.y - other.y, self.z - other.z)
        else:
            raise TypeError("Can only subtract another Quaternion object.")

    def __mul__(self, other):
        """
        Multiplication of two quaternions.
        """
        if isinstance(other, Quaternion):
            w = self.w * other.w - self.x * other.x - self.y * other.y - self.z * other.z
            x = self.w * other.x + self.x * other.w + self.y * other.z - self.z * other.y
            y = self.w * other.y - self.x * other.z + self.y * other.w + self.z * other.x
            z = self.w * other.z + self.x * other.y - self.y * other.x + self.z * other.w
            return Quaternion(w, x, y, z)
        elif isinstance(other, (int, float)):
            return Quaternion(self.w * other, self.x * other, self.y * other, self.z * other)
        else:
            raise TypeError("Can only multiply by another Quaternion object or a scalar.")

    def __rmul__(self, other):
        """
        Right multiplication (for scalar multiplication).
        """
        return self.__mul__(other)

    def conjugate(self):
        """
        Returns the conjugate of the quaternion.
        """
        return Quaternion(self.w, -self.x, -self.y, -self.z)

    def norm(self):
        """
        Returns the norm (magnitude) of the quaternion.
        """
        return np.sqrt(self.w**2 + self.x**2 + self.y**2 + self.z**2)

    def unit(self):
        """
        Returns the unit quaternion (normalized).
        """
        norm_val = self.norm()
        if norm_val == 0:
            return Quaternion(0, 0, 0, 0)
        return Quaternion(self.w / norm_val, self.x / norm_val, self.y / norm_val, self.z / norm_val)

    def inverse(self):
        """
        Returns the inverse of the quaternion.
        """
        norm_sq = self.w**2 + self.x**2 + self.y**2 + self.z**2
        if norm_sq == 0:
            raise ZeroDivisionError("Cannot find the inverse of a zero quaternion.")
        conjugate_q = self.conjugate()
        return Quaternion(conjugate_q.w / norm_sq, conjugate_q.x / norm_sq, conjugate_q.y / norm_sq, conjugate_q.z / norm_sq)

    def rotate_vector(self, vector):
        """
        Rotates a 3D vector by the quaternion.

        Args:
            vector (list or tuple): A 3-element list or tuple representing the vector [x, y, z].

        Returns:
            numpy.ndarray: The rotated vector as a NumPy array.
        """
        if not isinstance(vector, (list, tuple)) or len(vector) != 3:
            raise ValueError("Input must be a 3D vector (list or tuple).")

        v = Quaternion(0, vector[0], vector[1], vector[2])
        rotated_v = (self * v * self.conjugate()).imaginary_part()
        return np.array([rotated_v.x, rotated_v.y, rotated_v.z])

    def imaginary_part(self):
        """
        Returns the imaginary part of the quaternion as a new Quaternion object.
        """
        return Quaternion(0, self.x, self.y, self.z)

Example usage
```python
if __name__ == "__main__":
    q1 = Quaternion(1, 2, 3, 4)
    q2 = Quaternion(0, 1, -1, 0.5)
    scalar = 2.5
    vector = [1, 0, 0]

    print("Quaternion q1:", q1)
    print("Quaternion q2:", q2)

    print("\nAddition (q1 + q2):", q1 + q2)
    print("Subtraction (q1 - q2):", q1 - q2)
    print("Multiplication (q1 * q2):", q1 * q2)
    print("Scalar multiplication (q1 * scalar):", q1 * scalar)
    print("Scalar multiplication (scalar * q1):", scalar * q1)

    print("\nConjugate of q1:", q1.conjugate())
    print("Norm of q1:", q1.norm())
    print("Unit quaternion of q1:", q1.unit())
    print("Inverse of q2:", q2.inverse())

    print("\nRotating vector", vector, "by q2:")
    rotated_vector = q2.rotate_vector(vector)
    print("Rotated vector:", rotated_vector)
```

In [ ]:
P0 = (( 1,  0, 0, 0.3740), 
      ( 0,  1, 0, 0     ), 
      ( 0,  0, 1, 0.6300), 
      ( 0,  0, 0, 1     ))
P1 = (( 0,  0, 1, 0.3038), 
      ( 0,  1, 0, 0     ), 
      (-1,  0, 0, 0.0510),
      ( 0,  0, 0, 1     ))
#P2 = ((0, -1, 0, 0     ), (0, 0, 1, 0.3020), (-1, 0, 0, 0.5580), (0, 0, 0, 1))
tau = 1
T = 10

In [ ]:
x, y, z

In [ ]:
def interp(P0, P1, lambd):
    # interpolate the position
    ix = P0(0)
    # interpolate the orientation